## Calculating Impact Score

Calculating an impact score encompassing property tax change, median income change, # of new construction units, and # of zoning appeals or variances. The impact score will use data from 2015-2023 so it will measure impact on a 8 year timeline.

In [2]:
import pandas as pd
from pandas import json_normalize
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import json

### Property Tax Change - IGNORE

In [ ]:
base_url = "https://phl.carto.com/api/v2/sql"
query = """
SELECT *, ST_AsGeoJson(the_geom)::json AS the_geom_geojson 
FROM assessments
LIMIT 
"""
params = {
    "q": query
}
results = requests.get(base_url, params)

# Check the status code
print(f"Status code: {results.status_code}")

# Look at the first part of the response content
print(f"First 200 characters of response: {results.text[:200]}")

# Check the content length
print(f"Content length: {len(results.text)}")

Status code: 200
First 200 characters of response: {"rows":[{"cartodb_id":1,"the_geom":null,"the_geom_webmercator":null,"parcel_number":"011000001","year":2025,"market_value":148100,"taxable_land":148100,"taxable_building":0,"exempt_land":0,"exempt_bu
Content length: 11230052


In [91]:
parcel = gpd.read_file(r"C:\Penn MCP\Fourth Semester\Practicum\notebooks\Philadelphia_DOR_Parcels_202402.shp")

In [92]:
parcel.head()

,PARCELID,TENCODE,ADDRESS,OWNER1,OWNER2,BLDG_DESC,IMPERV_ARE,BC_LANDUSE,BC_TYPE,IMP_ROOF,...,NATURAL_GR,TOTAL_GROU,COUNCIL_DI,PCPC_DISTR,PUBLIC_PRI,PROGRAM,OWNER_CATE,Shape__Are,Shape__Len,geometry
0,139435,8797004855,4855 N 9TH ST,PHEONIX PROPERTIES LLC,NaN,ROW B/GAR 2STY MASONRY,1355,Building,Other,1091.97,...,180.33,444.04,Council District 8,Upper North,PRIVATE,RESIDENTIAL,RESIDENTIAL_UNDER_4_UNITS,243.648438,87.387180,"POLYGON ((-8364355.928 4869766.602, -8364356.9..."
1,140629,1308008421,8421 ARDLEIGH ST,MC NALLY HUGH,CHRISTINE H/W,SEMI/DET 3 STY MAS.+OTHER,1570,Building,Other,819.01,...,1223.18,1974.73,Council District 8,Upper Northwest,PRIVATE,RESIDENTIAL,RESIDENTIAL_UNDER_4_UNITS,443.828125,111.627374,"POLYGON ((-8371752.506 4877133.351, -8371780.1..."
2,139436,8805004817,4817 N 13TH ST,BRYANT EBONY A,NaN,SEMI DET 2 STY MASONRY,1283,Building,Other,1260.53,...,362.76,385.59,Council District 8,Upper North,PRIVATE,RESIDENTIAL,RESIDENTIAL_UNDER_4_UNITS,261.121094,83.633419,"POLYGON ((-8365083.950 4869769.945, -8365084.6..."
3,141829,2228000405,405 W CHAMPLOST AVE,MAM SAVY,NaN,ROW B/GAR 2STY MASONRY,953,Building,Other,715.80,...,69.80,307.34,Council District 9,Upper North,PRIVATE,RESIDENTIAL,RESIDENTIAL_UNDER_4_UNITS,162.371094,67.602728,"POLYGON ((-8363158.048 4872143.688, -8363153.7..."
4,139438,6712005119,5119 RACE ST,STRANGE REGINALD,NaN,ROW 2 STY MASONRY,941,Building,Other,941.34,...,228.01,228.01,Council District 3,West,PRIVATE,RESIDENTIAL,RESIDENTIAL_UNDER_4_UNITS,185.148438,70.571661,"POLYGON ((-8373776.918 4860599.140, -8373781.7..."


In [95]:
parcel.columns

Index(['PARCELID', 'TENCODE', 'ADDRESS', 'OWNER1', 'OWNER2', 'BLDG_DESC',
       'IMPERV_ARE', 'BC_LANDUSE', 'BC_TYPE', 'IMP_ROOF', 'IMP_GROUND',
       'IMP_TOTAL', 'NATURAL_GR', 'TOTAL_GROU', 'COUNCIL_DI', 'PCPC_DISTR',
       'PUBLIC_PRI', 'PROGRAM', 'OWNER_CATE', 'Shape__Are', 'Shape__Len',
       'geometry'],
      dtype='object')

In [66]:
print("Count of each year:")
print(prop['year'].value_counts())

Count of each year:
year
2024    571860
2023      9856
2025      2249
2022         3
2021         1
Name: count, dtype: int64


In [72]:
prop_filtered = prop[(prop['year'] >= 2015) & (prop['year'] <= 2023)]

# Calculate total taxable value (sum of taxable land and building)
prop_filtered['taxable_value'] = prop_filtered['taxable_land'] + prop_filtered['taxable_building']

# Check if a property is tax-exempt (both taxable land and building are 0)
prop_filtered['is_exempt'] = prop_filtered['taxable_value'] == 0

# Philadelphia Real Estate Tax rates by year (from the official document)
tax_rates = {
    2015: 0.01340,   # 1.340%
    2016: 0.013998,  # 1.3998%
    2017: 0.013998,  # 1.3998%
    2018: 0.013998,  # 1.3998%
    2019: 0.013998,  # 1.3998%
    2020: 0.013998,  # 1.3998% (same as 2019 based on document)
    2021: 0.013998,  # 1.3998% (same as 2019 based on document)
    2022: 0.013998,  # 1.3998% (same as 2019 based on document)
    2023: 0.013998   # 1.3998% (same as 2019 based on document)
}

# Calculate property tax for all properties
prop_filtered['property_tax'] = prop_filtered.apply(
    lambda row: row['taxable_value'] * tax_rates.get(row['year'], 0), 
    axis=1
)

# Filter to get only taxable properties
taxable_prop = prop_filtered[~prop_filtered['is_exempt']]


In [41]:
taxable_prop = taxable_prop[['year', 'geometry', 'property_tax']]
taxable_prop.head()

,year,geometry,property_tax
577,2023,POINT (-75.17148 40.03862),744.69360
668,2023,POINT (-75.21408 40.05827),111.98400
705,2023,POINT (-75.24731 39.96853),2697.41460
783,2023,POINT (-74.97560 40.09672),4115.41200
820,2023,POINT (-75.24075 39.97146),1150.70559


In [55]:
rco = gpd.read_file(r"C:\Penn MCP\Fourth Semester\Practicum\notebooks\Zoning_RCO.shp")

In [57]:
print("RCO CRS:", rco.crs)
print("Taxable Prop CRS:", taxable_prop.crs)

# Set CRS for construction data (assuming it's in WGS84 - EPSG:4326, which is common for GeoJSON)
taxable_prop.set_crs(epsg=4326, inplace=True)

# Now reproject to match RCO's CRS
taxable_prop = taxable_prop.to_crs(rco.crs)

# Perform spatial join
taxable_prop_rco = gpd.sjoin(rco, taxable_prop, how='right', predicate='intersects')
taxable_prop_rco = taxable_prop_rco[['year', 'geometry', 'property_tax', 'ORGANIZATI']]
taxable_prop_rco = taxable_prop_rco.rename(columns={'year': 'assessment_year', 'ORGANIZATI': 'RCO'})
taxable_prop_rco.head()

RCO CRS: EPSG:4326
Taxable Prop CRS: EPSG:4326


,assessment_year,geometry,property_tax,RCO
577,2023,POINT (-75.17148 40.03862),744.6936,Faith Community Development Corporation (FCDC)
577,2023,POINT (-75.17148 40.03862),744.6936,12th Ward Democratic Committee
577,2023,POINT (-75.17148 40.03862),744.6936,KECO INC
577,2023,POINT (-75.17148 40.03862),744.6936,Wingohocking & Wister Neighbors
668,2023,POINT (-75.21408 40.05827),111.9840,Chestnut Hill Conservancy


In [58]:
districts = gpd.read_file('https://opendata.arcgis.com/api/v3/datasets/1ba5a5d68f4a4c75806e78b1d9245924_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1')

In [60]:
taxable_prop_rco_dis = gpd.sjoin(taxable_prop_rco, districts, how='left', predicate='intersects')
taxable_prop_rco_dis = taxable_prop_rco_dis[['assessment_year', 'geometry', 'property_tax', 'RCO', 'DISTRICT']]
taxable_prop_rco_dis.head()

,assessment_year,geometry,property_tax,RCO,DISTRICT
577,2023,POINT (-75.17148 40.03862),744.6936,Faith Community Development Corporation (FCDC),8
577,2023,POINT (-75.17148 40.03862),744.6936,12th Ward Democratic Committee,8
577,2023,POINT (-75.17148 40.03862),744.6936,KECO INC,8
577,2023,POINT (-75.17148 40.03862),744.6936,Wingohocking & Wister Neighbors,8
668,2023,POINT (-75.21408 40.05827),111.9840,Chestnut Hill Conservancy,8


### New Construction

In [3]:
import geopandas as gpd
import fiona
import json
from shapely.geometry import shape

In [4]:
def read_geojson(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        geojson_data = json.load(file)
    return geojson_data

file_path = r"C:\Penn MCP\Fourth Semester\Practicum\notebooks\philadelphia_construction.geojson"
gdf = read_geojson(file_path)

In [5]:
gdf

{'type': 'FeatureCollection',
 'name': 'philadelphia_construction',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'parcel_id_num': -888680028,
    'conscomplete': '2023-10-28T18:56:30Z',
    'the_geom': '0101000020E6100000EE04A385C7CB52C0D288FD3389F44340',
    'completion_year': 2023},
   'geometry': {'type': 'Point',
    'coordinates': [-75.18405285758828, 39.91043710593807]}},
  {'type': 'Feature',
   'properties': {'parcel_id_num': -888660469,
    'conscomplete': '2022-10-13T18:56:53Z',
    'the_geom': '0101000020E61000002F3201A4E8BE52C0186A615B7E0B4440',
    'completion_year': 2022},
   'geometry': {'type': 'Point',
    'coordinates': [-74.98294925801086, 40.08979360825123]}},
  {'type': 'Feature',
   'properties': {'parcel_id_num': -888300924,
    'conscomplete': '2021-07-15T15:00:49Z',
    'the_geom': '0101000020E610000057A6CC43CBCB52C0D0ECD9C4D6F84340',
    'completion_year': 2021},
   'ge

In [6]:
# Convert the GeoJSON data to a GeoDataFrame
cons = gpd.GeoDataFrame.from_features(gdf['features'])

In [7]:
cons.head()

,geometry,parcel_id_num,conscomplete,the_geom,completion_year
0,POINT (-75.18405 39.91044),-888680028,2023-10-28T18:56:30Z,0101000020E6100000EE04A385C7CB52C0D288FD3389F4...,2023
1,POINT (-74.98295 40.08979),-888660469,2022-10-13T18:56:53Z,0101000020E61000002F3201A4E8BE52C0186A615B7E0B...,2022
2,POINT (-75.18428 39.94405),-888300924,2021-07-15T15:00:49Z,0101000020E610000057A6CC43CBCB52C0D0ECD9C4D6F8...,2021
3,POINT (-75.18428 39.94405),-888300920,2021-03-24T15:49:57Z,0101000020E610000057A6CC43CBCB52C0D0ECD9C4D6F8...,2021
4,POINT (-75.18428 39.94405),-888300804,2023-04-03T16:44:42Z,0101000020E610000057A6CC43CBCB52C0D0ECD9C4D6F8...,2023


In [8]:
rco = gpd.read_file(r"C:\Penn MCP\Fourth Semester\Practicum\notebooks\Zoning_RCO.shp")

In [9]:
rco.head()

,OBJECTID,ORGANIZATI,ORGANIZA_1,MEETING_LO,ORG_TYPE,PREFFERED_,PRIMARY_NA,PRIMARY_AD,PRIMARY_EM,PRIMARY_PH,...,ALTERNAT_1,ALTERNAT_2,ALTERNAT_3,A_PHONE_EX,EXPIRATION,EFFECTIVE_,LNI_ID,Shape__Are,Shape__Len,geometry
0,1,Girard Estate Neighbors Association,"PO Box 20116\r\nPhiladelphia, pA 19145","Donatucci Library\r\n1935 W Shunk St., Philade...",Other,Email,Jessica Frye,"2527 South Cleveland Street\r\nPhiladelphia, P...",jlynnfrye@gmail.com,2158706166,...,"2527 South Cleveland Street\r\nPhiladelphia, P...",flowershow1997@yahoo.com,2674089341,NaN,2025,2014-03-01,203,1.082550e+06,4463.667453,"POLYGON ((-75.18689 39.91886, -75.18757 39.919..."
1,2,SoLo/Germantown Civic Association,"4908 Wayne Ave Philadelhia, PA 19144",5058 Wayne Ave (in person suspended) Phila PA ...,Other,Email,Allison Weiss,"4908 Wayne Ave Philadelphia, PA 19144",awfromhh6@gmail.com,2158435555,...,"4941 Royal St Philadelphia, PA 19144",\t\tthekeithbrooks@hotmail.com,2022711494,NaN,2025,2014-03-01,321,5.993613e+06,11871.164642,"POLYGON ((-75.16022 40.02130, -75.16059 40.020..."
2,3,East Passyunk Crossing Civic Association (EPX),"1025 Mifflin St Philadelphia, PA 19148","1025 Mifflin St Philadelphia, PA 19148",Other,Email,Bruce Benjamin,"1430 Walnut Street 3rd Floor Philadelphia, PA...",Zoning@epcrossing.org,2158050172,...,"1120 Morris Street Philadelphia, PA 19148",ajsciolla@yahoo.com,2158808514,NaN,2025,2014-03-01,174,1.318087e+06,4734.886361,"POLYGON ((-75.16626 39.93022, -75.16559 39.930..."
3,4,Residents of Shawmont Valley Association,18 Shawmont Ave,"8230 Ridge Avenue Philadelphia, PA 19128",Other,Email,Aaron Couch,"399 Port Royal Ave.\r\nPhiladelphia, PA 19128",shawmontvalley@gmail.com,NaN,...,"21 River Road \r\nPhiladelphia, Pennsylvania 1...",robertsmall1@comcast.net,2154367594,NaN,2025,2014-03-01,311,4.730749e+06,9316.543271,"POLYGON ((-75.25151 40.06573, -75.24160 40.057..."
4,5,Passyunk Square Civic Association,"PO Box 18052 Philadelphia, PA 19147",General Membership Meetings: South Philadelphi...,Other,Email,David Goldfarb,1168 S 13th St,dbgoldfarb@gmail.com,2155863975,...,"939 Federal Street\r\nPhiladelphia, PA 19147",suzanne.tavani@gmail.com,2159018132,NaN,2025,2014-03-01,289,1.537982e+06,4984.320226,"POLYGON ((-75.16671 39.93785, -75.16661 39.937..."


In [10]:
# Check CRS of both datasets
print("RCO CRS:", rco.crs)
print("Construction CRS:", cons.crs)

# Set CRS for construction data (assuming it's in WGS84 - EPSG:4326, which is common for GeoJSON)
cons.set_crs(epsg=4326, inplace=True)

# Now reproject to match RCO's CRS
cons = cons.to_crs(rco.crs)

# Perform spatial join
construction_rco = gpd.sjoin(rco, cons, how='right', predicate='intersects')


RCO CRS: EPSG:4326
Construction CRS: None


In [11]:
construction_rco.head()

,index_left,OBJECTID,ORGANIZATI,ORGANIZA_1,MEETING_LO,ORG_TYPE,PREFFERED_,PRIMARY_NA,PRIMARY_AD,PRIMARY_EM,...,EXPIRATION,EFFECTIVE_,LNI_ID,Shape__Are,Shape__Len,geometry,parcel_id_num,conscomplete,the_geom,completion_year
0,74.0,75.0,Friends of Historic FDR Park,OFFICIAL ADDRESS FOR DELIVERIES IS 1900 PATTIS...,SWEDISH MUSEUM - IN THE PARK - 1900 PATTISON A...,Other,Email,BARBARA CAPOZZI,3310 SOUTH 20TH STREET \r\nPHILA PA 19145,BCPACKERPARK@GMAIL.COM,...,2025.0,2014-03-14,193.0,4.380927e+06,9279.814182,POINT (-75.18405 39.91044),-888680028,2023-10-28T18:56:30Z,0101000020E6100000EE04A385C7CB52C0D288FD3389F4...,2023
0,78.0,79.0,Packer Park Civic Association,"3310 SOUTH 20TH ST\r\nC/O BARBARA CAPOZZI, ESQ...",CITY LIFE CHURCH / GYM \r\n3301 SOUTH 20TH STR...,Other,Email,Barbara Capozzi,"3310 South 20th Street\r\nPhila, Pa 19145",BCPACKERPARK@GMAIL.COM,...,2025.0,2014-03-14,282.0,1.235200e+07,14992.514981,POINT (-75.18405 39.91044),-888680028,2023-10-28T18:56:30Z,0101000020E6100000EE04A385C7CB52C0D288FD3389F4...,2023
0,132.0,133.0,26th Republican Ward,"1713 Wolf St 1st Fl, Philadelphia PA 19145","1713 Wolf St, Philadelphia PA 19145",Ward,Email,George E. Gunning,"PO Box 20136\r\nPhiladelphia, PA 19145",ggunning@26thward.org,...,2026.0,2022-07-08,26.0,2.107706e+07,20882.266417,POINT (-75.18405 39.91044),-888680028,2023-10-28T18:56:30Z,0101000020E6100000EE04A385C7CB52C0D288FD3389F4...,2023
0,183.0,184.0,Friends of Penrose,"2541 South Broad Street, Philadelphia, PA 19148","2020 Penrose Avenue, Philadelphia, PA 19148",Other,Email,Vincent DeFino,"2541 South Broad Street, Philadelphia, PA 19148",dvino2@aol.com,...,2025.0,NaN,195.0,2.056223e+06,6552.489486,POINT (-75.18405 39.91044),-888680028,2023-10-28T18:56:30Z,0101000020E6100000EE04A385C7CB52C0D288FD3389F4...,2023
1,146.0,147.0,Northeast Community Civic Alliance,"9605 Leon Street Philadelphia\r\nPhiladelphia,...",9551 Academy Rd,Other,Email,Thomas Neveil,"9605 Leon Street\r\nPhiladelphia, PA 19114",thomas.neveil@gmail.com,...,2025.0,2014-06-25,265.0,4.540015e+07,31814.134135,POINT (-74.98295 40.08979),-888660469,2022-10-13T18:56:53Z,0101000020E61000002F3201A4E8BE52C0186A615B7E0B...,2022


In [12]:
construction_rco = construction_rco[['ORGANIZATI', 'geometry', 'completion_year']]

In [13]:
construction_rco.head()

,ORGANIZATI,geometry,completion_year
0,Friends of Historic FDR Park,POINT (-75.18405 39.91044),2023
0,Packer Park Civic Association,POINT (-75.18405 39.91044),2023
0,26th Republican Ward,POINT (-75.18405 39.91044),2023
0,Friends of Penrose,POINT (-75.18405 39.91044),2023
1,Northeast Community Civic Alliance,POINT (-74.98295 40.08979),2022


In [14]:
construction_rco = construction_rco.rename(columns={
    'ORGANIZATI': 'RCO',
    'completion_year': 'cons_complete'
})

construction_rco.head()

,RCO,geometry,cons_complete
0,Friends of Historic FDR Park,POINT (-75.18405 39.91044),2023
0,Packer Park Civic Association,POINT (-75.18405 39.91044),2023
0,26th Republican Ward,POINT (-75.18405 39.91044),2023
0,Friends of Penrose,POINT (-75.18405 39.91044),2023
1,Northeast Community Civic Alliance,POINT (-74.98295 40.08979),2022


### Add Districts and Join Datasets

In [15]:
districts = gpd.read_file('https://opendata.arcgis.com/api/v3/datasets/1ba5a5d68f4a4c75806e78b1d9245924_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1')

In [16]:
joined = gpd.sjoin(construction_rco, districts, how='left', predicate='intersects')
joined.head()

,RCO,geometry,cons_complete,index_right,OBJECTID_1,OBJECTID,DISTRICT,SHAPE_LENG,Shape__Area,Shape__Length
0,Friends of Historic FDR Park,POINT (-75.18405 39.91044),2023,1.0,2.0,2.0,2,163242.842040,9.866933e+07,64896.883391
0,Packer Park Civic Association,POINT (-75.18405 39.91044),2023,1.0,2.0,2.0,2,163242.842040,9.866933e+07,64896.883391
0,26th Republican Ward,POINT (-75.18405 39.91044),2023,1.0,2.0,2.0,2,163242.842040,9.866933e+07,64896.883391
0,Friends of Penrose,POINT (-75.18405 39.91044),2023,1.0,2.0,2.0,2,163242.842040,9.866933e+07,64896.883391
1,Northeast Community Civic Alliance,POINT (-74.98295 40.08979),2022,9.0,10.0,10.0,10,129145.536207,1.055363e+08,51487.825562


In [17]:
joined = joined[['RCO', 'geometry', 'cons_complete', 'DISTRICT']]

joined = joined.rename(columns={
    'geometry': 'new_cons_geo'
})
joined.head()

,RCO,new_cons_geo,cons_complete,DISTRICT
0,Friends of Historic FDR Park,POINT (-75.18405 39.91044),2023,2
0,Packer Park Civic Association,POINT (-75.18405 39.91044),2023,2
0,26th Republican Ward,POINT (-75.18405 39.91044),2023,2
0,Friends of Penrose,POINT (-75.18405 39.91044),2023,2
1,Northeast Community Civic Alliance,POINT (-74.98295 40.08979),2022,10


### Change of Use

### Census - Median Property Value & Income 

I decided to do the census data analysis in R!

In [18]:
census = gpd.read_file("C:\Penn MCP\Fourth Semester\Practicum\philly_acs.gpkg")

census.head()

,GEOID,med_income_2015,med_income_2016,med_income_2017,med_income_2018,med_income_2019,med_income_2020,med_income_2021,med_income_2022,med_income_2023,med_home_value_2015,med_home_value_2016,med_home_value_2017,med_home_value_2018,med_home_value_2019,med_home_value_2020,med_home_value_2021,med_home_value_2022,med_home_value_2023,geometry
0,421010190001,37829.0,41692.0,43134.0,43529.0,NaN,NaN,NaN,58438.0,61083.0,89400.0,94200.0,91100.0,86400.0,NaN,83400.0,151000.0,NaN,166700.0,"MULTIPOLYGON (((-75.10315 40.00620, -75.10283 ..."
1,421010191003,35093.0,27292.0,31458.0,33750.0,36517.0,63839.0,43432.0,46986.0,45345.0,89100.0,72400.0,83700.0,87400.0,91000.0,121400.0,NaN,168000.0,203600.0,"MULTIPOLYGON (((-75.11181 40.01366, -75.10965 ..."
2,421010197003,16211.0,NaN,NaN,8266.0,NaN,NaN,NaN,NaN,NaN,65000.0,68700.0,95800.0,63700.0,85500.0,81600.0,66900.0,75600.0,NaN,"MULTIPOLYGON (((-75.13878 40.01868, -75.13846 ..."
3,421010199001,23295.0,22978.0,19630.0,18750.0,19323.0,10884.0,14857.0,12266.0,NaN,43800.0,49600.0,47400.0,45000.0,41500.0,NaN,81200.0,91200.0,67300.0,"MULTIPOLYGON (((-75.14099 40.00553, -75.14083 ..."
4,421010200001,NaN,29231.0,NaN,36736.0,35379.0,35702.0,11273.0,14654.0,NaN,36100.0,64400.0,63100.0,64100.0,66900.0,65300.0,NaN,82700.0,92500.0,"MULTIPOLYGON (((-75.15372 39.99799, -75.15368 ..."


In [19]:
joined.set_geometry('new_cons_geo', inplace=True)

In [20]:
census.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [21]:
print("joined CRS:", joined.crs)
print("census CRS:", census.crs)

census.set_crs(epsg=4326, inplace=True, allow_override=True)

census = census.to_crs(joined.crs)

joined CRS: EPSG:4326
census CRS: EPSG:4269


In [22]:
census.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [23]:
yimby_data = gpd.sjoin(joined, census, how='left', predicate='intersects')
yimby_data.head()

,RCO,new_cons_geo,cons_complete,DISTRICT,index_right,GEOID,med_income_2015,med_income_2016,med_income_2017,med_income_2018,...,med_income_2023,med_home_value_2015,med_home_value_2016,med_home_value_2017,med_home_value_2018,med_home_value_2019,med_home_value_2020,med_home_value_2021,med_home_value_2022,med_home_value_2023
0,Friends of Historic FDR Park,POINT (-75.18405 39.91044),2023,2,767.0,421010373002,80625.0,92969.0,95852.0,92321.0,...,116875.0,348700.0,353900.0,352900.0,360700.0,368000.0,353700.0,364500.0,418600.0,371900.0
0,Packer Park Civic Association,POINT (-75.18405 39.91044),2023,2,767.0,421010373002,80625.0,92969.0,95852.0,92321.0,...,116875.0,348700.0,353900.0,352900.0,360700.0,368000.0,353700.0,364500.0,418600.0,371900.0
0,26th Republican Ward,POINT (-75.18405 39.91044),2023,2,767.0,421010373002,80625.0,92969.0,95852.0,92321.0,...,116875.0,348700.0,353900.0,352900.0,360700.0,368000.0,353700.0,364500.0,418600.0,371900.0
0,Friends of Penrose,POINT (-75.18405 39.91044),2023,2,767.0,421010373002,80625.0,92969.0,95852.0,92321.0,...,116875.0,348700.0,353900.0,352900.0,360700.0,368000.0,353700.0,364500.0,418600.0,371900.0
1,Northeast Community Civic Alliance,POINT (-74.98295 40.08979),2022,10,615.0,421010362022,69750.0,61328.0,50889.0,58750.0,...,87169.0,209300.0,202500.0,210300.0,213600.0,215700.0,115500.0,NaN,NaN,NaN


In [24]:
yimby_data = yimby_data.drop('index_right', axis=1)

yimby_data.head()

,RCO,new_cons_geo,cons_complete,DISTRICT,GEOID,med_income_2015,med_income_2016,med_income_2017,med_income_2018,med_income_2019,...,med_income_2023,med_home_value_2015,med_home_value_2016,med_home_value_2017,med_home_value_2018,med_home_value_2019,med_home_value_2020,med_home_value_2021,med_home_value_2022,med_home_value_2023
0,Friends of Historic FDR Park,POINT (-75.18405 39.91044),2023,2,421010373002,80625.0,92969.0,95852.0,92321.0,88322.0,...,116875.0,348700.0,353900.0,352900.0,360700.0,368000.0,353700.0,364500.0,418600.0,371900.0
0,Packer Park Civic Association,POINT (-75.18405 39.91044),2023,2,421010373002,80625.0,92969.0,95852.0,92321.0,88322.0,...,116875.0,348700.0,353900.0,352900.0,360700.0,368000.0,353700.0,364500.0,418600.0,371900.0
0,26th Republican Ward,POINT (-75.18405 39.91044),2023,2,421010373002,80625.0,92969.0,95852.0,92321.0,88322.0,...,116875.0,348700.0,353900.0,352900.0,360700.0,368000.0,353700.0,364500.0,418600.0,371900.0
0,Friends of Penrose,POINT (-75.18405 39.91044),2023,2,421010373002,80625.0,92969.0,95852.0,92321.0,88322.0,...,116875.0,348700.0,353900.0,352900.0,360700.0,368000.0,353700.0,364500.0,418600.0,371900.0
1,Northeast Community Civic Alliance,POINT (-74.98295 40.08979),2022,10,421010362022,69750.0,61328.0,50889.0,58750.0,69375.0,...,87169.0,209300.0,202500.0,210300.0,213600.0,215700.0,115500.0,NaN,NaN,NaN


In [30]:
yimby_data['income_pct_change'] = (yimby_data['med_income_2023'] - yimby_data['med_income_2015']) / yimby_data['med_income_2015']
yimby_data['home_value_pct_change'] = (yimby_data['med_home_value_2023'] - yimby_data['med_home_value_2015']) / yimby_data['med_home_value_2015']

# Step 2: Create impact_score as average of the two (or weight them if you prefer)
yimby_data['impact_score'] = (yimby_data['income_pct_change'] + yimby_data['home_value_pct_change']) / 2

In [32]:
yimby_data

,RCO,new_cons_geo,cons_complete,DISTRICT,GEOID,med_income_2015,med_income_2016,med_income_2017,med_income_2018,med_income_2019,...,med_home_value_2018,med_home_value_2019,med_home_value_2020,med_home_value_2021,med_home_value_2022,med_home_value_2023,raw_impact_score,impact_score,income_pct_change,home_value_pct_change
0,Friends of Historic FDR Park,POINT (-75.18405 39.91044),2023,2,421010373002,80625.0,92969.0,95852.0,92321.0,88322.0,...,360700.0,368000.0,353700.0,364500.0,418600.0,371900.0,25.807262,0.258073,0.449612,0.066533
0,Packer Park Civic Association,POINT (-75.18405 39.91044),2023,2,421010373002,80625.0,92969.0,95852.0,92321.0,88322.0,...,360700.0,368000.0,353700.0,364500.0,418600.0,371900.0,25.807262,0.258073,0.449612,0.066533
0,26th Republican Ward,POINT (-75.18405 39.91044),2023,2,421010373002,80625.0,92969.0,95852.0,92321.0,88322.0,...,360700.0,368000.0,353700.0,364500.0,418600.0,371900.0,25.807262,0.258073,0.449612,0.066533
0,Friends of Penrose,POINT (-75.18405 39.91044),2023,2,421010373002,80625.0,92969.0,95852.0,92321.0,88322.0,...,360700.0,368000.0,353700.0,364500.0,418600.0,371900.0,25.807262,0.258073,0.449612,0.066533
1,Northeast Community Civic Alliance,POINT (-74.98295 40.08979),2022,10,421010362022,69750.0,61328.0,50889.0,58750.0,69375.0,...,213600.0,215700.0,115500.0,NaN,NaN,NaN,NaN,NaN,0.249735,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23609,South Port Richmond Civic Association,POINT (-75.10905 39.98482),2025,1,421010180023,41463.0,41684.0,58630.0,62031.0,67778.0,...,123700.0,128500.0,145800.0,149000.0,163800.0,184500.0,92.190110,0.921901,1.311410,0.532392
23609,New Kensington Community Development Corp,POINT (-75.10905 39.98482),2025,1,421010180023,41463.0,41684.0,58630.0,62031.0,67778.0,...,123700.0,128500.0,145800.0,149000.0,163800.0,184500.0,92.190110,0.921901,1.311410,0.532392
23610,Port Richmond On Patrol & Civic Association (P...,POINT (-75.10914 39.98477),2025,1,421010180023,41463.0,41684.0,58630.0,62031.0,67778.0,...,123700.0,128500.0,145800.0,149000.0,163800.0,184500.0,92.190110,0.921901,1.311410,0.532392
23610,South Port Richmond Civic Association,POINT (-75.10914 39.98477),2025,1,421010180023,41463.0,41684.0,58630.0,62031.0,67778.0,...,123700.0,128500.0,145800.0,149000.0,163800.0,184500.0,92.190110,0.921901,1.311410,0.532392


In [33]:
import os

os.makedirs("output_data", exist_ok=True)

yimby_data.to_file(os.path.join("output_data", "output_file.geojson"), driver="GeoJSON")